In [ ]:
%reload_ext autoreload
%autoreload 2
import zeus.notebook_utils.syspath as syspath
syspath.add_parent_folder()

In [ ]:
from typing import Tuple
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from kidney.utils.plotting import overlay
from kidney.datasets.kaggle import get_reader, SampleType
from zeus.plotting.utils import calculate_layout

In [ ]:
THUMBNAIL_SIZE = 1024, 1024

In [ ]:
reader = get_reader()

In [ ]:
keys = reader.get_keys(SampleType.Labeled)

In [ ]:
def detect_roi(image: np.ndarray, thumbnail_size: Tuple[int, int] = THUMBNAIL_SIZE) -> Tuple[int, int, int, int]:
    thumbnail = cv.resize(image, thumbnail_size)
    gray = cv.cvtColor(thumbnail, cv.COLOR_BGR2GRAY)
    wt, ht = thumbnail_size
    thresh = cv.threshold(gray, 0, 255, cv.THRESH_OTSU + cv.THRESH_BINARY)[1]
    contours = cv.findContours(thresh, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    contours = contours[0] if len(contours) else contours[1]
    contours = sorted(contours, key=cv.contourArea, reverse=True)
    biggest = contours[0]
    xr, yr, wr, hr = cv.boundingRect(biggest)
    w0, h0 = image.shape[1], image.shape[0]
    wt, ht = thumbnail_size
    rw, rh = w0/wt, h0/ht
    x0, y0, x1, y1 = [int(v) for v in [
        (xr * rw),
        (yr * rh),
        (xr + wr) * rw,
        (yr + hr) * rh
    ]]
    return x0, y0, x1, y1

In [ ]:
n, m = calculate_layout(len(keys), n_cols=2)

f, axes = plt.subplots(n, m, figsize=(m * 8, n * 8))

for ax in axes.flat:
    ax.set_visible(False)

for i, key in enumerate(keys):
    image = reader.fetch_one(key)['image']
    ax = axes.flat[i]
    ax.set_visible(True)
    x0, y0, x1, y1 = detect_roi(image)
    thumbnail = cv.resize(image[y0:y1, x0:x1, :], THUMBNAIL_SIZE)
    ax.imshow(thumbnail)